In [1]:
import herringbone as hb

All initialization tests passed.
imported herringbone without any errors :)


In [2]:

# create an MDP
map_names = ["-", "example", "easy", "danger_holes", "double_fish", "wall_of_death", "example2", "mega"]
selected_map_id = 1

state_path = "herringbone/env_core/config/state_config.json"
map_path = f"herringbone/env_core/maps/{map_names[selected_map_id]}.csv"
action_path = "herringbone/env_core/config/action_config.json"

GAMMA = 0.9

demo_mdp = hb.MDP(state_path, map_path, action_path, seed=42, gamma=GAMMA)


In [ ]:

random_policy = hb.EpsilonGreedyPolicy(mdp=demo_mdp, epsilon=1)
episode = hb.Episode(mdp=demo_mdp, policy=random_policy, max_depth=1000)
episode.peek()
episode.run("ascii")
print(episode.trajectory)

╔═══════╦═══════╦═══════╦═══════╦═══════╗
║   -1  ║   -1  ║   -1  ║   -1  ║   -1  ║
╠═══════╬═══════╬═══════╬═══════╬═══════╣
║   -1  ║   -1  ║   -1  ║   -1  ║   -1  ║
╠═══════╬═══════╬═══════╬═══════╬═══════╣
║   -1  ║   -1  ║   -1  ║   -1  ║   10  ║
╚═══════╩═══════╩═══════╩═══════╩═══════╝[0]
t: 0 | S: [2, 0], R: -1, A: ←
t: 1 | S: [2, 0], R: -1, A: ↓
t: 2 | S: [2, 0], R: -1, A: ↑
t: 3 | S: [1, 0], R: -1, A: ↑
t: 4 | S: [0, 0], R: -1, A: ↓
t: 5 | S: [1, 0], R: -1, A: ↑
t: 6 | S: [0, 0], R: -1, A: →
t: 7 | S: [0, 1], R: -1, A: ←
t: 8 | S: [0, 0], R: -1, A: ↑
t: 9 | S: [0, 0], R: -1, A: ↓
t: 10 | S: [1, 0], R: -1, A: ←
t: 11 | S: [1, 0], R: -1, A: ↓
t: 12 | S: [2, 0], R: -1, A: ←
t: 13 | S: [2, 0], R: -1, A: →
t: 14 | S: [2, 1], R: -1, A: ←
t: 15 | S: [2, 0], R: -1, A: ↑
t: 16 | S: [1, 0], R: -1, A: ↑
t: 17 | S: [0, 0], R: -1, A: →
t: 18 | S: [0, 1], R: -1, A: ←
t: 19 | S: [0, 0], R: -1, A: ←
t: 20 | S: [0, 0], R: -1, A: ↑
t: 21 | S: [0, 0], R: -1, A: ↓
t: 22 | S: [1, 0], R: -1, A: ↑


In [4]:
# Add the render function for previewing a Value Function (hate this!)
def preview_V(mdp, learned_V):
    
    states_2d = mdp.get_board().states
    x = len(states_2d)
    y = len(states_2d[0])
    
    v_values =  ['%.2f' % v for v in list(learned_V.values())]

    two_d_list = [v_values[i * y:(i + 1) * y] for i in range(x)]

    # Print 2D list
    for row in two_d_list:
        print(row)

In [5]:
policy = hb.EpsilonGreedyPolicy(mdp=demo_mdp, epsilon=1)  # defaults to uniform
print(policy)

╔═════════╦═════════╦═════════╦═════════╦═════════╗
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╚═════════╩═════════╩═════════╩═════════╩═════════╝


In [6]:
episode = hb.Episode(mdp=demo_mdp, policy=policy, max_depth=1000)

In [7]:
episode.run("sar")

t: 0 | S: [2, 0], R: -1, A: ↓
t: 1 | S: [2, 0], R: -1, A: ↑
t: 2 | S: [1, 0], R: -1, A: ↑
t: 3 | S: [0, 0], R: -1, A: ↓
t: 4 | S: [1, 0], R: -1, A: →
t: 5 | S: [1, 1], R: -1, A: →
t: 6 | S: [1, 2], R: -1, A: →
t: 7 | S: [1, 3], R: -1, A: ↑
t: 8 | S: [0, 3], R: -1, A: ↑
t: 9 | S: [0, 3], R: -1, A: ←
t: 10 | S: [0, 2], R: -1, A: ←
t: 11 | S: [0, 1], R: -1, A: →
t: 12 | S: [0, 2], R: -1, A: ↑
t: 13 | S: [0, 2], R: -1, A: ←
t: 14 | S: [0, 1], R: -1, A: ↓
t: 15 | S: [1, 1], R: -1, A: ↑
t: 16 | S: [0, 1], R: -1, A: ←
t: 17 | S: [0, 0], R: -1, A: ↓
t: 18 | S: [1, 0], R: -1, A: ↓
t: 19 | S: [2, 0], R: -1, A: ↑
t: 20 | S: [1, 0], R: -1, A: →
t: 21 | S: [1, 1], R: -1, A: ←
t: 22 | S: [1, 0], R: -1, A: ←
t: 23 | S: [1, 0], R: -1, A: ↓
t: 24 | S: [2, 0], R: -1, A: ↑
t: 25 | S: [1, 0], R: -1, A: →
t: 26 | S: [1, 1], R: -1, A: ↓
t: 27 | S: [2, 1], R: -1, A: →
t: 28 | S: [2, 2], R: -1, A: ↓
t: 29 | S: [2, 2], R: -1, A: →
t: 30 | S: [2, 3], R: -1, A: ↓
t: 31 | S: [2, 3], R: -1, A: →
t: 32 | S: [2, 4],

In [8]:
N = 1000
mc_predictor = hb.MonteCarloPredictor(demo_mdp)
mc_predictor.evaluate_policy(policy, n_samples=N)

In [9]:
preview_V(demo_mdp, mc_predictor.value_functions)

['-8.66', '-8.16', '-6.98', '-5.40', '-3.63']
['-8.56', '-7.95', '-6.70', '-3.94', '0.23']
['-8.39', '-7.59', '-5.62', '-1.01', '0.00']


In [10]:

N = 1000
mc_control = hb.MonteCarloController(demo_mdp, epsilon=0.25)
mc_control.train(n_episodes=N)
trained_policy = mc_control.policy

In [11]:
print(trained_policy.get_policy())
print(trained_policy)

{[0, 0]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [0, 1]: {↑: 0.0625, ↓: 0.8125, ←: 0.0625, →: 0.0625}, [0, 2]: {↑: 0.0625, ↓: 0.8125, ←: 0.0625, →: 0.0625}, [0, 3]: {↑: 0.06249999999981622, ↓: 0.8124999999962357, ←: 0.06250000000350547, →: 0.0625000000004426}, [0, 4]: {↑: 0.06172818508349649, ↓: 0.8067586212797855, ←: 0.0667464836478563, →: 0.06476670998886169}, [1, 0]: {↑: 0.0625, ↓: 0.8125, ←: 0.0625, →: 0.0625}, [1, 1]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [1, 2]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [1, 3]: {↑: 0.0625, ↓: 0.8125, ←: 0.0625, →: 0.0625}, [1, 4]: {↑: 0.0625, ↓: 0.8125, ←: 0.0625, →: 0.0625}, [2, 0]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [2, 1]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [2, 2]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [2, 3]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [2, 4]: {↑: 0.25, ↓: 0.25, ←: 0.25, →: 0.25}}
╔═════════╦═════════╦═════════╦═════════╦═════════╗
║    →    ║    ↓    ║    ↓    ║    ↓    ║    

In [12]:
episode = hb.Episode(mdp=demo_mdp, policy=trained_policy, max_depth=1000)
episode.run("sar")

t: 0 | S: [2, 0], R: -1, A: ←
t: 1 | S: [2, 0], R: -1, A: →
t: 2 | S: [2, 1], R: -1, A: →
t: 3 | S: [2, 2], R: -1, A: ↑
t: 4 | S: [1, 2], R: -1, A: ↑
t: 5 | S: [0, 2], R: -1, A: ↓
t: 6 | S: [1, 2], R: -1, A: →
t: 7 | S: [1, 3], R: -1, A: ↓
t: 8 | S: [2, 3], R: -1, A: →
t: 9 | S: [2, 4], R: 10, A: None


In [13]:
N = 1000
mc_predictor = hb.MonteCarloPredictor(demo_mdp)
mc_predictor.evaluate_policy(trained_policy, n_samples=N)

In [14]:
preview_V(demo_mdp, mc_predictor.value_functions)

['0.48', '1.93', '3.45', '4.81', '8.00']
['1.29', '3.03', '4.88', '6.80', '9.51']
['2.53', '4.38', '6.55', '9.21', '0.00']
